In [3]:
#!pip install pyg-nightly tabulate torch sklearn sentencepiece transformers sentence-transformers pandas faiss-cpu numpy folium streamlit datasets pickle5 sklearn hazm nltk tqdm networkx==2.6.3 spacy en_core_web_sm

In [1]:
import socket, time
from http.server import BaseHTTPRequestHandler, HTTPServer, SimpleHTTPRequestHandler, test
from urllib.parse import parse_qs
import json
import pandas as pd
import sentencepiece
import tqdm
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
import re
from transformers import AutoTokenizer, BigBirdModel, DistilBertForSequenceClassification, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import confusion_matrix
from datasets import load_metric
from transformers import pipeline
import torch
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.cluster import KMeans
nltk.download("popular")
from transformers import AutoConfig, AutoTokenizer, AutoModel, TFAutoModel
import faiss
import pickle
%load_ext autoreload
from sentence_transformers import SentenceTransformer
from pathlib import Path
from gensim.models import FastText
from random import shuffle
from __future__ import unicode_literals
import random
import sys
import codecs
from nltk import FreqDist
import itertools
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
import networkx as nx
from tabulate import tabulate

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/ImanAlipour/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/ImanAlipour/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/ImanAlipour/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/ImanAlipour/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/ImanAlipour/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /Users/ImanAlipour/nltk_data...
[nltk_data]    |   Pa

# HW4
## HW4 data

In [10]:
df_hw4 = pd.read_csv('https://github.com/AmooHashem/my-datasets/raw/main/sentiment140.csv')
df_hw4.columns = ['pos_or_neg', 'id', 'date', 'query', 'tweeter', 'passage']
df_hw4['pos_or_neg'].replace({4: 1}, inplace=True)

In [11]:
# str_ = '    @im i\'m.    very @!!ha.ppy.  to# see_ you @nf yes!    ' # Test
max_dataset_items = 10000
df_hw4 = df_hw4.sample(n=max_dataset_items)

def normalize_text(input_str):
  str_ = input_str.lower()
  str_ = re.sub('@[^\s]+ ', '', str_)
  str_ = re.sub('[^a-zA-Z0-9\s]', '', str_)
  str_ = str_.strip()
  str_ = re.sub('\s+', ' ', str_)
  return str_

stop_words = set(stopwords.words('english'))
stop_words.add('subject')
stop_words.add('http')

def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words])

lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

df_hw4['passage'] = df_hw4['passage'].apply(lambda x: normalize_text(x))
df_hw4['passage'] = df_hw4['passage'].apply(lambda x: remove_stopwords(x))
df_hw4['passage'] = df_hw4['passage'].apply(lambda text: lemmatize_words(text))

## Clasification
### A) Logistic Regression

In [24]:
vectorizer_hw4 = TfidfVectorizer(ngram_range=(1,2), stop_words = None, norm='l2')
x_tfidf_hw4 = vectorizer_hw4.fit_transform(df_hw4.passage)
x_train_lr, x_test_lr, y_train_lr, y_test_lr = train_test_split(x_tfidf_hw4, df_hw4.pos_or_neg, test_size = 0.1, random_state = 1)
regressor_hw4 = LogisticRegression(random_state=0, solver='lbfgs', max_iter=1000).fit(x_train_lr, y_train_lr)

def lr_results():
    metrics = []
    
    y_predicted = regressor_hw4.predict(x_test_lr)
    score = f1_score(y_test_lr, y_predicted, average='macro')

    metrics.append(f'logistic regression\'s f1_macro score: {score}')

    y_predicted = regressor_hw4.predict(x_test_lr)
    score = f1_score(y_test_lr, y_predicted, average='micro')

    metrics.append(f'logistic regression\'s f1_micro data score: {score}')

    results = confusion_matrix(y_test_lr, y_predicted)
    metrics.append(f'logistic regression\'s confusion matrix:')
    for row in results:
        metrics.append(f'{row}')
    

    y_test1 = [y for y in y_test_lr]
    y_pred1 = [y for y in y_predicted]
    counter = 0
    for i in range(len(y_test_lr)):
      if y_test1[i] == y_pred1[i]:
        counter += 1
    metrics.append(f'logistic regression\'s accuracy is: {100*counter/len(y_test_lr)}%')
    
    return metrics

lr_results()

["logistic regression's f1_macro score: 0.502292471042471",
 "logistic regression's f1_micro data score: 0.802",
 "logistic regression's confusion matrix:",
 '[789   1]',
 '[197  13]',
 "logistic regression's accuracy is: 80.2%"]

## B) Transformer

In [13]:
'''
MODEL_NAME = 'google/bigbird-base-trivia-itc'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
Y_hw4 = [y for y in df_hw4.pos_or_neg]
X_hw4 = [x for x in df_hw4.passage]
x_train_tr, x_testeval_tr, y_train_tr, y_testeval_tr = train_test_split(X_hw4, Y_hw4, test_size = 0.2, random_state = 1)
x_test_tr, x_eval_tr, y_test_tr, y_eval_tr = train_test_split(x_testeval_tr, y_testeval_tr, test_size = 0.5, random_state = 1)
train_encodings_tr = tokenizer([x for x in x_train_tr], truncation=True, padding=True)
eval_encodings_tr = tokenizer([x for x in x_eval_tr], truncation=True, padding=True)
test_encodings_tr = tokenizer([x for x in x_test_tr], truncation=True, padding=True)
class TweetDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings=encodings
    self.labels=labels
  
  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item
  
  def __len__(self):
    return len(self.labels)

train_dataset_tr = TweetDataset(train_encodings_tr, y_train_tr)
eval_dataset_tr = TweetDataset(eval_encodings_tr, y_eval_tr)
test_dataset_tr = TweetDataset(test_encodings_tr, y_test_tr)

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

c1_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

lr = 2e-5 
weight_decay = 0.001
num_epochs = 2
batch_size = 128
accumulation_steps = 4
num_workers = 10
max_len = 2048 
metric = load_metric('f1')

args = TrainingArguments(
    "Tweet-classifier",
    evaluation_strategy="epoch",
    logging_strategy='epoch',
    save_strategy="no",
    learning_rate=lr,
    label_smoothing_factor=0.05,
    weight_decay=weight_decay,
    dataloader_num_workers=num_workers,
    gradient_accumulation_steps=accumulation_steps,
    gradient_checkpointing=True,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    report_to='none',
    )

trainer = Trainer(
    model=c1_model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
    )

trainer.train()
trainer.evaluate(test_dataset)

generator = pipeline('text-classification', c1_model, tokenizer=tokenizer, config={'max_length':256})
y_predicted_transformer = [int(y['label'].split('_')[1]) for y in generator([x for x in x_test])]

def transformer_results():
    metrics = []
    metrics.append(f'transformer model\'s f1_macro score: {f1_score(y_test, y_predicted_transformer, average="macro")} ')
    results = confusion_matrix(y_test, y_predicted_transformer)
    metrics.append(f'transformer model\'s confusion matrix:')
    for row in results:
        metrics.append(f'{row}')
    counter = 0
    for i in range(len(y_test)):
      if y_test[i] == y_predicted_transformer[i]:
        counter += 1
    metrics.append(f'transformer model\'s  accuracy is: {100*counter/len(y_test)}%')
    
    return metrics

transformer_results()
'''

print()

## Clustering
### A) K-Means

In [14]:
X_cluster = [x for x in df_hw4['passage']]
vectorizer = TfidfVectorizer(stop_words='english')
X_cluster = vectorizer.fit_transform(X_cluster)

true_k = 5
clustering_model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
clustering_model.fit(X_cluster)

def get_l2_distance(vec1, vec2):
  return float(np.dot(vec1-vec2,  (vec1-vec2).T))

def calculate_RSS(kmeans_model):
  RSS = 0
  for point in X_cluster:
    min_dist = get_l2_distance(kmeans_model.cluster_centers_[0], point)
    for center in kmeans_model.cluster_centers_:
      if get_l2_distance(point, center) < min_dist:
        min_dist = get_l2_distance(point, center)
    
    RSS += min_dist/15
  
  return RSS

def clustering_results():
    metrics = []
    metrics.append(f'RSS of our clustering with k-means algorithm is: {calculate_RSS(clustering_model)}')

    clusters = [[], [], [], [], []]
    preds = [x for x in df_hw4["pos_or_neg"]]
    for i in range(1, len(preds)):
      clusters[clustering_model.labels_[i]].append(preds[i])

    true_elements = 0
    for cluster in clusters:
      counts = pd.Series(cluster).value_counts()
      true_elements += max(counts.get(0), counts.get(1))
    metrics.append(f'purity of our clustering with k-means algorithm is: {true_elements/len(preds)}')
    
    return metrics

clustering_results()

['RSS of our clustering with k-means algorithm is: 653.6047237872614',
 'purity of our clustering with k-means algorithm is: 0.8008']

# HW5

Importing data/preprocessing/using NER to get the named entities and generating the graph

In [15]:
require_cols = [0, 1, 8]

df_hw5 = pd.read_excel('https://github.com/AmooHashem/my-datasets/raw/main/%40khamenei_ir_user_tweets.xlsx', usecols = require_cols, engine='openpyxl')

DOC_NUMBER = len(df_hw5.index)

In [16]:
emoj = re.compile("["
        u"\U0001F600-\U0001F64F" 
        u"\U0001F300-\U0001F5FF" 
        u"\U0001F680-\U0001F6FF"  
        u"\U0001F1E0-\U0001F1FF"  
        u"\U00002500-\U00002BEF"  
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"
        u"\u3030"
                      "]+", re.UNICODE)
def remove_emojis(data):
    return re.sub(emoj, '', data)

lemmatizer_hw5 = WordNetLemmatizer()

def is_valid_word(text):
  return text[0] != '@' and re.search(r"[^\w]", text) is None

def normalize_word(text):
  text = re.sub(r'[\.:!،؛؟»\]\)\}«\[\(\{]','', text)
  text = remove_emojis(text)
  return text

def normalize_sent(text):
  sent = []
  for word in text.split():
    if is_valid_word(word):
      sent.append(normalize_word(word))
  sent = ' '.join(sent)
  sent = word_tokenize(sent)
  sent = ' '.join(sent)
  doc = nlp(sent)
  sent = [(X.text, X.label_) for X in doc.ents]
  return sent


data_hw5 = [df_hw5.values[i][1] for i in range(len(df_hw5.values))]
messages_hw5 = [x for x in data_hw5]
message_entities_hw5 = []
for message in tqdm.tqdm(messages_hw5):
  message_entities_hw5.append(normalize_sent(str(message)))

total_entities_hw5 = set()
for entities in tqdm.tqdm(message_entities_hw5):
  for entity in entities:
    total_entities_hw5.add(entity)

100%|███████████████████████████████████| 3167/3167 [00:00<00:00, 929556.39it/s]


In [17]:
G_hw5 = nx.Graph()
G_hw5.add_nodes_from(total_entities_hw5)
for entities in message_entities_hw5:
  for i in range(len(entities)):
    for j in range(i+1, len(entities)):
      G_hw5.add_edge(entities[i], entities[j])

def sort_tuples(tup):
  lst = len(tup)
  for i in range(0, lst):
    for j in range(0, lst-i-1):
      if (tup[j][1] < tup[j + 1][1]):
        temp = tup[j]
        tup[j]= tup[j + 1]
        tup[j + 1]= temp
  return tup

## Link Analysis
### A) Page Rank

In [18]:
pr_hw5 = nx.pagerank(G_hw5)
entity_prob_hw5 = []

for entity in total_entities_hw5:
  entity_prob_hw5.append((entity, pr_hw5[entity]))

categories_hw5 = ['NORP', 'ORG', 'DATE', 'PERSON', 'GPE', 'LOC', 'CARDINAL']
entity_prob_hw5 = sort_tuples(entity_prob_hw5)
entity_category_hw5 = {}
for entity in tqdm.tqdm(entity_prob_hw5):
  for category in categories_hw5:
    if entity[0][1] == category:
      if category not in entity_category_hw5:
        entity_category_hw5[category] = []
      if len(entity_category_hw5[category]) < 10:
        entity_category_hw5[category].append(entity[0][0])


100%|███████████████████████████████████| 1549/1549 [00:00<00:00, 604145.15it/s]


In [19]:
def page_rank_results():
    results = []
    for category in entity_category_hw5:
        results.append(category)
        results.append('_______')
        for entity in entity_category_hw5[category]:
            results.append(f'-------{entity}')

    return results


### A) HITS

In [20]:
def hits_results(num_outputs=10):
    results = []
    h_hw5, a_hw5 = nx.hits(G_hw5)
    hubs_hw5 = []
    for hub in h_hw5:
      hubs_hw5.append((hub, h_hw5[hub]))

    hubs_hw5 = sort_tuples(hubs_hw5)

    authorities_hw5 = []
    for authority in a_hw5:
      authorities_hw5.append((authority, a_hw5[authority]))

    authorities_hw5 = sort_tuples(authorities_hw5)

    results .append(f'Top {num_outputs} hubs: ')
    results .append('_____')
    for hub in hubs_hw5[:num_outputs]:
        results.append('(' + hub[0][0] + ', ' + hub[0][1] + f' ) with score: {hub[1]}')
        
    results.append('⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜')
    results.append('⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜')
    results .append(f'Top {num_outputs} authorities: ')
    results .append('_____')
    for authority in authorities_hw5[:num_outputs]:
        results.append('(' + authority[0][0] + ', ' + authority[0][1] + f' ) with score: {authority[1]}')
        
    return results


# Setting up the server

In [21]:
def generate_response(query, query_type, queryExp):
    if query_type == 'classification':
        return lr_results() #+ transformer_results()
    elif query_type == 'clustering':
        return clustering_results()
    elif query_type == 'hits':
        return hits_results(10)
    elif query_type == 'page_rank':
        return page_rank_results()

In [22]:
hostName = "localhost"
serverPort = 8080

In [ ]:
class MyServer(BaseHTTPRequestHandler):
    def end_headers (self):
        self.send_header('Access-Control-Allow-Origin', '*')
        SimpleHTTPRequestHandler.end_headers(self)
        
    def do_GET(self):
        self.send_response(200)
        self.send_header("Content-type", "text/html")
        self.end_headers()
        self.wfile.write(bytes("<html><head><title>https://pythonbasics.org</title></head>", "utf-8"))
        self.wfile.write(bytes("<p>Request: %s</p>" % self.path, "utf-8"))
        self.wfile.write(bytes("<body>", "utf-8"))
        self.wfile.write(bytes("<p>This is an example web server.</p>", "utf-8"))
        self.wfile.write(bytes("</body></html>", "utf-8"))
        
    def do_POST(self):
        # 1. How long was the message?
        length = int(self.headers.get('Content-length', 0))

        # 2. Read the correct amount of data from the request.
        data = self.rfile.read(length).decode()

        # 3. Extract the "message" field from the request data.
        message = json.loads(data)['query']
        print(json.loads(data))
        response = generate_response(json.loads(data)['query'], json.loads(data)['type'], json.loads(data)['queryExp'])
        reply = {'answers': response, 'expanded_query': 'I\'m an exmanded query and I\'m very very happy panda! 🐼  :D'}
        
        # Send the "message" field back as the response.        
        self.send_response(200)
        self.send_header('Content-type', 'text/plain; charset=utf-8')
        self.end_headers()
        self.wfile.write(json.dumps(reply).encode())

if __name__ == "__main__":        
    webServer = HTTPServer((hostName, serverPort), MyServer)
    print("Server started http://%s:%s" % (hostName, serverPort))

    try:
        webServer.serve_forever()
    except KeyboardInterrupt:
        pass

    webServer.server_close()
    print("Server stopped.")


Server started http://localhost:8080
{'query': '', 'queryExp': False, 'type': 'classification', 'mode': 'no-cors'}


127.0.0.1 - - [22/Jul/2022 12:57:18] "POST /classification/ HTTP/1.1" 200 -
